In [11]:
import numpy as np
import xarray as xr
import rasterio as rio
import rioxarray
from sklearn import linear_model
import matplotlib.pyplot as plt
import math
import pickle
from scipy import stats

## read in data

In [2]:
data_path = '/mnt/Backups/gbrench/repos/rg_uavsar/data'

path_dic = {'swatch_00540':f'{data_path}/swatch_00540/geo',
            'swatch_08301':f'{data_path}/swatch_08301/geo',
            'swatch_09510':f'{data_path}/swatch_09510/geo',
            'swatch_17304_s1':f'{data_path}/swatch_17304_s1/geo',
            'swatch_17304_s2':f'{data_path}/swatch_17304_s2/geo',
            'swatch_18525':f'{data_path}/swatch_18525/geo',
            'swatch_26300':f'{data_path}/swatch_26300/geo',
            'swatch_27518':f'{data_path}/swatch_27518/geo'}

In [3]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn, engine='h5netcdf', phony_dims='sort')
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                          'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [4]:
def open_uavsar(swatch, path):
    igram_ds = xr.open_dataset(f'{path}/{swatch}_20230725_20230925_unw_phase_m.tif').squeeze()
    cor_ds = xr.open_dataset(f'{path}/{swatch}_20230725_20230925_cor.tif').squeeze()
    geom_ds = mintpy2d_to_xarray(f'{path}/geo_geometryRadar.h5', crs='4326')
    geom_ds['cor'] = (('y', 'x'), cor_ds.band_data.values)
    geom_ds['displacement'] = (('y', 'x'), igram_ds.band_data.values)

    # reproject to utm
    geom_ds = geom_ds.rio.reproject('EPSG:32613', nodata=np.nan)

    # open rock glacier raster
    rgi_ds = xr.open_dataset(f'{data_path}/rgi_raster/rgi_raster.tif').squeeze()
    rgi_ds = rgi_ds.rio.reproject_match(geom_ds)

    geom_ds['rgi'] = (('y', 'x'), rgi_ds.band_data.values)
    
    return geom_ds

In [5]:
# open all data
ds_dic = {}
for key, item in path_dic.items():
    print(f'opening {key}')
    ds_dic[key] = open_uavsar(key, item)

opening swatch_00540
opening swatch_08301
opening swatch_09510
opening swatch_17304_s1
opening swatch_17304_s2
opening swatch_18525
opening swatch_26300
opening swatch_27518


## atmospheric correction

In [17]:
# functions to perform multiple reference point atmospheric correction
def select_refs(ds, corr_thresh, n_refs):
    ref_list = []
    ref_counter = 0
    while ref_counter < n_refs:
        x, y = np.random.randint(0, len(ds.x)), np.random.randint(0, len(ds.y))
        if [x, y] not in ref_list:
            if ds.cor.isel(x=x, y=y) >= corr_thresh:
            # add that ref point can't be in rock glacier body?
                ref_list.append([x, y])
                ref_counter +=1
    return ref_list

def select_refs_even_elev(ds, corr_thresh, n_refs, n_intervals=5):
    '''
    select n reference points evenly from i elevation intervals
    '''
    # define elevation intervals
    ref_list = []
    
    min_elev = ds.height.min().item()
    elev_range = (ds.height.max() - min_elev).item()
    interval_range = elev_range/n_intervals
    for i in range(0, n_intervals):
        interval_elev_min = (i*interval_range)+min_elev
        interval_elev_max = ((i+1)*interval_range)+min_elev

        # print(f'working on inverval {i+1}, {interval_elev_min}-{interval_elev_max}')
        interval_ref_counter = 0
        while interval_ref_counter < (n_refs/n_intervals):
            x, y = np.random.randint(0, len(ds.x)), np.random.randint(0, len(ds.y))
            if ds.height.isel(x=x, y=y) < interval_elev_max and ds.height.isel(x=x, y=y) > interval_elev_min:
                if [x, y] not in ref_list:
                    if ds.cor.isel(x=x, y=y) >= corr_thresh:
                        if ds.rgi.isel(x=x, y=y) != 1:
                            ref_list.append([x, y])
                            interval_ref_counter +=1
    return ref_list

def plot_first_scatter(ref_displacement, ref_elevation):
    f, ax = plt.subplots()
    ax.plot(ref_elevation, ref_displacement, marker='o', linestyle='')
    ax.set_xlabel('elevation')
    ax.set_ylabel('apparent displacement')
    ax.set_title('elevation and displacement of ref points')

def sample_refs(ds, refs):
    ref_displacement = []
    ref_elevation = []
    for ref in refs:
        ref_elevation.append(ds.height.isel(x=ref[0], y=ref[1]).item())
        ref_displacement.append(ds.displacement.isel(x=ref[0], y=ref[1]).item())
    return ref_displacement, ref_elevation

def linear_fit(ref_displacement, ref_elevation):
    model = linear_model.LinearRegression() 
    inputs = np.array((ref_elevation, ref_displacement)).transpose()
    inputs = inputs[~np.isnan(inputs).any(axis=1)]
    model.fit(inputs[:, 0].reshape(-1, 1), inputs[:, 1])
    fit = [model.coef_.item(), model.intercept_]
    return fit

def polynomial_fit(ref_displacement, ref_elevation, order=2):
    inputs = np.array((ref_elevation, ref_displacement)).transpose()
    inputs = inputs[~np.isnan(inputs).any(axis=1)]
    fit = np.polyfit(inputs[:, 0], inputs[:, 1], order)
    return fit

def nmad(data):
    """Compute the Normalized Median Absolute Deviation of data."""
    return 1.4826 * stats.median_abs_deviation(data, scale='normal')

def binned_polynomial_fit(ref_displacement, ref_elevation, order=2, bin_width=100):
    # Remove rows with NaN values
    inputs = np.array((ref_elevation, ref_displacement)).transpose()
    inputs = inputs[~np.isnan(inputs).any(axis=1)]
    
    # Define bin edges with a width of bin_width
    bin_edges = np.arange(inputs[:, 0].min(), inputs[:, 0].max(), bin_width)

    # Digitize the dependent variable, i.e., assign each y to a bin
    bins = np.digitize(inputs[:, 0], bin_edges)

    # Initialize arrays for medians and nmads
    medians = np.full(len(bin_edges) - 1, np.nan)
    nmads = np.full(len(bin_edges) - 1, np.nan)

    # Compute median and NMAD for each bin
    for i in range(1, len(bin_edges)):
        if np.any(bins == i):  # Check if bin is not empty
            if len(bin_data) > 1:
                medians[i-1] = np.median(inputs[:, 1][bins == i])
                nmads[i-1] = nmad(inputs[:, 1][bins == i])

    # Remove bins with NaN medians or nmads
    valid_bins = ~np.isnan(medians) & ~np.isnan(nmads)
    bin_medians = medians[valid_bins]
    bin_nmads = nmads[valid_bins]
    bin_centers = bin_edges[:-1][valid_bins] + bin_width / 2

    # Fit the polynomial to the bin medians, weighted by the inverse of the bin NMADs
    fit = np.polyfit(bin_centers, bin_medians, order, w=1/bin_nmads)

    return fit

def correct_igram(ds, fit):
    poly = np.poly1d(fit)
    ds['displacement_MuRP'] = ds.displacement - poly(ds.height)
    # for linear fit: 
    #slope, intercept = fit
    #ds['displacement_MuRP'] = ds.displacement - (ds.height*slope+intercept)
    return ds

# single function
def MuRP(ds, corr_thresh=0.95, n_refs=1000):
    '''
    Correct unwrapped phase with linear fit to multiple stable reference points
    '''
    # print('selecting reference points')
    # refs = select_refs(ds, corr_thresh, n_refs)
    refs = select_refs_even_elev(ds, corr_thresh, n_refs)
    
    # print('sampling reference points')
    ref_displacement, ref_elevation = sample_refs(ds, refs)
    # plot_first_scatter(ref_displacement, ref_elevation)
    
    # print('calculating fits')
    fit = binned_polynomial_fit(ref_displacement, ref_elevation)
    
    # print('correcting interferogram')
    ds = correct_igram(ds, fit)
    correction_data = [ref_displacement, ref_elevation, fit]

    return ds, correction_data

In [9]:
# tiled atmospheric correction
def tiled_correction(ds, tile_size=1400, skip_size=700):
    xmin=0
    xmax=tile_size
    ymin=0
    ymax=tile_size

    correction_data = []
    
    corrected_array = np.full(ds.displacement.shape, np.nan)

    #loop through tiles
    for i in range(math.ceil((len(ds.x)/skip_size))):
        print(f'column {i}')
        for j in range(math.ceil((len(ds.y)/skip_size))):
            print(f'row {j}')
            ymin = j*skip_size
            ymax = ymin+tile_size
            xmin = i*skip_size
            xmax = xmin+tile_size

            tile_ds = ds.isel(x=slice(xmin, xmax), y=slice(ymin, ymax))
            
            # skip tiles that don't include valid data
            if tile_ds.displacement.count() == 0:
                continue

            # adjust tile to include sufficient data
            while tile_ds.displacement.count() < (skip_size**2):
                # if too few values, expand tile size by n pixels in each direction
                if ymax < len(ds.y)/2:
                    ymax += skip_size
                else: 
                    ymin -= skip_size
                if xmax < len(ds.x)/2:
                    xmax += skip_size
                else:
                    xmin -= skip_size

                tile_ds = ds.isel(x=slice(xmin, xmax), y=slice(ymin, ymax))

            print(f'tile extent x: {xmin}, {xmax}, y: {ymin}, {ymax}')

            corrected_ds, tile_correction_data = MuRP(tile_ds)
            corrected_tile = np.full(ds.displacement.shape, np.nan)
            corrected_tile[ymin:ymax, xmin:xmax] = corrected_ds.displacement_MuRP.values

            # add tile to corrected array, taking mean where data overlaps
            corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)

            # preserve correction data in list
            correction_data.append(tile_correction_data)

    ds['displacement_MuRP'] = (('y', 'x'), corrected_array)

    return ds, correction_data

In [18]:
# correct all interferograms
for key, ds in ds_dic.items():
    print(f'working on {key}')
    ds_corrected, correction_data = tiled_correction(ds)
    pickle.dump(correction_data, open(f'{path_dic[key]}/correction_data.pkl', "wb"))
    ds_corrected.displacement_MuRP.rio.to_raster(f'{path_dic[key]}/{key}_20230725_20230925_displacement_MuRP.tif', compress="LZW")
    ds_corrected.to_netcdf(f'{path_dic[key]}/{key}_20230725_20230925.nc')

working on swatch_00540
column 0
row 0
tile extent x: 0, 1400, y: 0, 1400


/tmp/ipykernel_26419/1782889505.py:49: RuntimeWarning: Mean of empty slice
  corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)


row 1
tile extent x: 0, 1400, y: 700, 2100


/tmp/ipykernel_26419/1782889505.py:49: RuntimeWarning: Mean of empty slice
  corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)


row 2
tile extent x: 0, 1400, y: 1400, 2800


...

/tmp/ipykernel_26419/1782889505.py:49: RuntimeWarning: Mean of empty slice
  corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)


row 5
tile extent x: 7700, 9800, y: 2800, 4900


/tmp/ipykernel_26419/1782889505.py:49: RuntimeWarning: Mean of empty slice
  corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)


row 6
tile extent x: 7700, 9800, y: 3500, 5600


/tmp/ipykernel_26419/3363805308.py:101: RankWarning: Polyfit may be poorly conditioned
  fit = np.polyfit(bin_centers, bin_medians, order, w=1/bin_nmads)
/tmp/ipykernel_26419/1782889505.py:49: RuntimeWarning: Mean of empty slice
  corrected_array = np.nanmean(np.stack([corrected_array, corrected_tile]), axis=0)
